In [1]:
import matplotlib.pyplot as plt
try:
    import IPython
    shell = IPython.get_ipython()
    shell.enable_matplotlib(gui='qt')
except:
    pass


from MJOLNIR import _tools
from MJOLNIR.Data import DataSet 
from lmfit import Model, Parameter, report_fit
from MJOLNIR.Data import Mask
from MJOLNIR._tools import fileListGenerator

import numpy as np
from os import path
from scipy.ndimage import gaussian_filter
from scipy import interpolate
import pandas as pd
import copy
from matplotlib.colors import LinearSegmentedColormap
import matplotlib 
from generateBG import generateBG
from fitting_funcs import * 

datdir = 'Data/'

In [5]:
#Subset of data files for elastic scattering
files = _tools.fileListGenerator("3827,3828,3830,3831,\
                                    3833,3834,3835",datdir,2023)
ds0 = DataSet.DataSet(files)
ds0.convertDataFile(binning = 8,saveFile=False) 

files = _tools.fileListGenerator("3881,3886,3887,3888,\
                                3878,3879,3880",datdir,2023)
ds11 = DataSet.DataSet(files)
ds11.convertDataFile(binning = 8,saveFile=False) 


mask=[]
#ds=ds
mask.append(Mask.indexMask(23,24,axis=1))
#Mask for Currate Axes spurions (Q points in [] and width given in dqx, dqy)
#mask.append(Mask.CurratAxeMask([[0,2,0]],dqx=0.1,dqy=0.23))
#mask.append(Mask.CurratAxeMask([[1,1,0],[-1,1,0]],dqx=0.23,dqy=0.1))
#Al line
mask.append(Mask.circleMask([0,0],[1.35,0],coordinates=['qx','qy'],maskInside=False)*Mask.circleMask([0,0],[1.45,0],coordinates=['qx','qy'],maskInside=True)*
            Mask.lineMask(4.1,4.6,coordinates='energy',maskInside=True)*Mask.lineMask(7.2,8,coordinates='Ei',maskInside=True))
mask.append(Mask.circleMask([0,0],[1.835,0],coordinates=['qx','qy'],maskInside=False)*Mask.circleMask([0,0],[1.9,0],coordinates=['qx','qy'],maskInside=True)*
            Mask.lineMask(4.1,4.6,coordinates='energy',maskInside=True)*Mask.lineMask(7.2,8,coordinates='Ei',maskInside=True))
mask.append(Mask.circleMask([0,0],[1.25,0],coordinates=['qx','qy'],maskInside=False)*Mask.circleMask([0,0],[1.3,0],coordinates=['qx','qy'],maskInside=True)*
            Mask.lineMask(4.1,4.,coordinates='energy',maskInside=True)*Mask.lineMask(7.2,8,coordinates='Ei',maskInside=True))
mask.append(Mask.circleMask([0,0],[1.5,0],coordinates=['qx','qy'],maskInside=False)*Mask.circleMask([0,0],[1.625,0],coordinates=['qx','qy'],maskInside=True)*
            Mask.lineMask(5.9,6.5,coordinates='energy',maskInside=True)*Mask.lineMask(9,9.5,coordinates='Ei',maskInside=True))


ds0.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds0.mask,np.sum(mask)(ds0))]
ds11.mask = [np.logical_or(m1,m2) for m1,m2 in zip(ds11.mask,np.sum(mask)(ds11))]

#Normalize to b / eV / sr / U
ds0.absoluteNormalize(sampleMass=1.1,sampleChemicalFormula="UTe2",
                       formulaUnitsPerUnitCell=4)
ds11.absoluteNormalize(sampleMass=1.1,sampleChemicalFormula="UTe2",
                       formulaUnitsPerUnitCell=4)

dssub = ds11-ds0


# Elastic Slice integrated in [-0.1,0.1] meV

In [6]:
# Const E slice 
astar = 2.0*np.pi/ds0[0].sample.a
bstar = 2.0*np.pi/ds0[0].sample.b
dset=ds0
EMin =-0.15 #Energy minimum
EMax =0.15 #Energy maximum
deltaE = np.abs(EMin - EMax)*1e-3 # 1e-3 to convert from meV to eV
#xBinTolerance = 2*np.pi/ds[0].sample.c*.03 #value in rlu. binning along x
#yBinTolerance = np.sqrt(5)*2*np.pi/ds[0].sample.b*0.03 #value in rlu
xBinTolerance = .02 #value in 1/A. binning along x
yBinTolerance = .02 #value in 1/A. binning along y

xBinTolerance = .02 #value in 1/A. binning along x
yBinTolerance = .02 #value in 1/A. binning along y
vmin=0 #minimum of the color scale
vmax=2.5#maximum of the color scale
cmap='Spectral_r' #cmap for smoothed plane




Data,ax = ds0.plotQPlane(EMin=EMin, EMax=EMax,xBinTolerance=xBinTolerance,
                          yBinTolerance=yBinTolerance,log=False,cmap = cmap,colorbar=1)

H,K = np.meshgrid(np.unique(np.around(Data['H'],3)),
                      np.unique(np.around(Data['K'],3)))
I = np.reshape(Data['Int'],np.shape(H.T))
I[I==-1.00e0]=np.nan
I*=deltaE
plt.close()

Data11,ax = ds11.plotQPlane(EMin=EMin, EMax=EMax,xBinTolerance=xBinTolerance,
                          yBinTolerance=yBinTolerance,log=False,cmap = cmap,colorbar=1)

H11,K11 = np.meshgrid(np.unique(np.around(Data11['H'],3)),
                      np.unique(np.around(Data11['K'],3)))
I11 = np.reshape(Data11['Int'],np.shape(H11.T))
I11[I11==-1.00e0]=np.nan
plt.close()
I11*=deltaE

DataSub,ax = dssub.plotQPlane(EMin=EMin, EMax=EMax,xBinTolerance=xBinTolerance,
                          yBinTolerance=yBinTolerance,log=False,cmap = cmap,colorbar=1)

HSub,KSub = np.meshgrid(np.unique(np.around(DataSub['H'],3)),
                      np.unique(np.around(DataSub['K'],3)))
ISub = np.reshape(DataSub['Int'],np.shape(HSub.T))
ISub[ISub==-1.00e0]=np.nan
ISub*=deltaE

plt.close()

fig,axs = plt.subplots(3,1,constrained_layout=True,figsize=(3.54,6)) # Extract figure from returned axis
mesh_a = axs[0].pcolormesh(H,K,I.T,vmin=vmin,vmax=vmax,cmap=cmap,rasterized=True)
cbar_a = fig.colorbar(mesh_a,label="I (b/sr/U)")
mesh_b = axs[1].pcolormesh(H11,K11,I11.T,vmin=vmin,vmax=vmax,cmap=cmap,rasterized=True)
cbar_b = fig.colorbar(mesh_b,label="I (b/sr/U)")

mesh_c = axs[2].pcolormesh(HSub,KSub,-ISub.T,vmin=-vmax/50,vmax=vmax/50,cmap='coolwarm',rasterized=True)
cbar_c = fig.colorbar(mesh_c,label="I (b/sr/U)")

axs[0].set_aspect(bstar/astar)
axs[1].set_aspect(bstar/astar)
axs[2].set_aspect(bstar/astar)

axs[0].set_xlabel(r"$(h00)$ (r.l.u.)")
axs[1].set_xlabel(r"$(h00)$ (r.l.u.)")
axs[2].set_xlabel(r"$(h00)$ (r.l.u.)")

axs[0].set_ylabel(r"$(0k0)$ (r.l.u.)")
axs[1].set_ylabel(r"$(0k0)$ (r.l.u.)")
axs[2].set_ylabel(r"$(0k0)$ (r.l.u.)")

#fig.colorbar(mesh_a,location='top') # Create colorbar from plot
#fig.colorbar(mesh_b,location='top')
ax.set_clim(vmin,vmax)
#ax.set_ylim(1.2,1.8)
#ax.set_xlim(-0.3,0.3)
axs[0].set_title(r"0 T $\hbar\omega\in$"+f"{EMin,EMax} meV")
axs[1].set_title(r"11 T $\hbar\omega\in$"+f"{EMin,EMax} meV")
axs[2].set_title(r"0 T - 11 T $\hbar\omega\in$"+f"{EMin,EMax} meV")
fig.savefig("Figures/Elastic_UTe2_slices.pdf",bbox_inches='tight')

## Transverse cuts along Bragg peaks to look for field-induced magnetic moment

In [7]:
import lmfit
from lmfit import Model,Parameters
from lmfit.models import GaussianModel,LinearModel

def get_trapz_err(x,errs,xlim=False):
    #Given the x values, errors, and limits of a trapzoidal integral returns the error bar of the 
    # result that would be given by np.trapz
    if xlim==False:
        xlim=[np.nanmin(x)-0.1,np.nanmax(x)+0.1]
    integral=0
    int_err=0
    good_i = np.intersect1d(np.where(x>=xlim[0])[0],np.where(x<=xlim[1])[0])
    x=x[good_i]
    errs=errs[good_i]
    for i in range(len(errs)-1):
        delX = np.abs(x[0]-x[i+1])
        term=delX**2 * (errs[i]**2 + errs[i+1]*2)/4.0
        int_err+=np.sqrt(term)
    return int_err

In [8]:
#Cuts showing mosaic, putting limits on short range ordering.

fig,ax = plt.subplots(3,2,figsize=(6,5),constrained_layout=True)
Qpeaks = [[0,2,0],[1,1,0],[-1,1,0]]
dQ_list = [[0.1,0,0],[0.09,-0.15,0],[-0.09,-0.15,0]]
sf_peaks =[]#Calculated elsewhere. 
for i,Qpt in enumerate(Qpeaks):
    dQ = np.array(dQ_list[i])
    Q1 = np.array(Qpt)-dQ
    Q2 = np.array(Qpt)+dQ
    dQint = 0.15
    Qstep =0.02
    Emin,Emax = -0.15,0.15
    deltaE = np.abs(Emin-Emax)
    scalefac = deltaE*1*1e-3
    data_0T_hdir = ds0.cut1D(Q1,Q2,dQint,Qstep,Emin,Emax)
    data_11T_hdir = ds11.cut1D(Q1,Q2,dQint,Qstep,Emin,Emax)
    data_sub_hdir = dssub.cut1D(Q1,Q2,dQint,Qstep,Emin,Emax)



    I = data_0T_hdir[0]['Int']*scalefac
    Err = data_0T_hdir[0]['Int_err']*scalefac
    Q=data_0T_hdir[0]['H']

    I11 = data_11T_hdir[0]['Int']*scalefac
    Err11 = data_11T_hdir[0]['Int_err']*scalefac
    Q11=data_11T_hdir[0]['H']

    weights = 1.0/Err
    weights[np.isnan(I)]=0

    model = GaussianModel()+LinearModel()
    #model = GaussianModel()+GaussianModel(prefix='g2_')+LinearModel()

    pars = model.make_params()
    pars.add('amplitude',value=np.nanmax(I),min=0,max=1e10)
    pars.add('center',value=Q[np.argmax(I)],min=np.nanmin(Q),max=np.nanmax(Q))
    pars.add('sigma',value=0.2,min=0.001,max=1)
    pars.add('slope',value=0,vary=False)
    pars.add('intercept',value=np.nanmin(I),min=0,max=np.nanmax(I))

    result= model.fit(I,x=Q,weights=weights,params=pars,method='powell')
    #print(result.fit_report())

    ax[i,0].errorbar(data_0T_hdir[0]['H'],data_0T_hdir[0]['Int']*scalefac,data_0T_hdir[0]['Int_err']*scalefac,
                    capsize=3,ls=' ',mfc='w',mec='k',color='k',marker='o')
    ax[i,0].errorbar(data_11T_hdir[0]['H'],data_11T_hdir[0]['Int']*scalefac,data_11T_hdir[0]['Int_err']*scalefac,
                    capsize=3,ls=' ',mfc='w',mec='b',color='b',marker='o')
    ax[i,0].plot(Q,result.best_fit,'r-')
    diff = data_sub_hdir[0]['Int']*scalefac
    differr = np.sqrt(Err**2 + Err11**2)
    #differr = data_sub_hdir[0]['Int_err']

    ax[i,1].errorbar(data_0T_hdir[0]['H'],diff,differr,
                    capsize=3,ls=' ',mfc='w',mec='k',color='k',marker='o')
    ax[i,1].plot(np.linspace(-3,3,1000),np.zeros(1000),'k--')
    ax[i,1].set_xlim(np.nanmin(Q),np.nanmax(Q))
    if i==2:
        ax[i,0].set_xlabel(r"(h00) (r.l.u.)")
        ax[i,1].set_xlabel(r"(h00) (r.l.u.)")
    ax[i,0].set_ylabel(r"I (b/sr/U)")
    diff_int = np.trapz(x=Q,y=diff)
    diff_int_err = np.nanmean(differr/diff)*diff_int
    ax[i,1].text(0.5,0.98,r"$\int I(Q)dQ=$"+f"\n{diff_int:.1e}({diff_int_err:.1e})",verticalalignment='top',
                 horizontalalignment='center',transform=ax[i,1].transAxes)
    ax[i,0].text(0.05,0.95,f"{Qpt}",verticalalignment='top',horizontalalignment='left',
                 transform=ax[i,0].transAxes)
    ax[i,1].set_ylim(np.nanmin(diff*1.2),np.nanmax(diff)*3)
fig.savefig('Figures/Elastic_cuts.pdf',bbox_inches='tight')

C:\Users\tjh\AppData\Local\anaconda3\envs\mjolnir\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\tjh\AppData\Local\anaconda3\envs\mjolnir\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\tjh\AppData\Local\anaconda3\envs\mjolnir\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
